In [1]:
import ligpy_utils as lig

In [2]:
T = 300

In [3]:
reactionlist, rateconstantlist, compositionlist = lig.set_paths()
y_list = lig.get_specieslist(reactionlist)

In [4]:
speciesindices, indices_to_species = lig.get_speciesindices(y_list)
rate_list = lig.build_rates_list(rateconstantlist, reactionlist, speciesindices, indices_to_species, T)
species_rxns = lig.build_species_rxns_dict(reactionlist)
dydt_expressions = lig.build_dydt_list(rate_list, y_list, species_rxns)
kmatrix = lig.build_k_matrix(rateconstantlist)
kvaluelist = lig.get_k_value_list(T, kmatrix)

In [8]:
with open('solve_ODEs_T%s.py' %T, 'w') as f:
    text = "#!/usr/bin/pythons\n" \
            "# -*- coding: utf-8 -*-\n\n" \
            "from scipy.integrate import odeint\n\n\n" \
            "def ODEs(y, t, p):\n"
    f.write(text)
    
    y = '\t' + y_list[0]
    for i in y_list[1:]:
        y += (', ' + i)
    y += ' = y'
    f.write(y + '\n')
    
    p = '\tk_%s_0' %T
    for i in range(1, len(kvaluelist)):
        p += ', k_%s_%s' %(T, i)
    p += ' = p'
    f.write(p + '\n')
    
    dydt = '\tdydt = [%s' %dydt_expressions[0].split('=')[1][1:-2]
    for ODE in dydt_expressions[1:]:
        dydt += (', \n\t\t\t' + ODE.split('=')[1][1:-2])
    dydt += ']'
    f.write(dydt + '\n')
    
    f.write('\treturn dydt\n\n')
    f.write('# k values\n')
    for i in range(len(kvaluelist)):
        f.write('k_%s_%s = %s\n' %(T, i, kvaluelist[i]))